### 웹 프레임워크 Django(python) 실습_window10환경

#### Contents
1. 프로젝트 & app 생성
2. 디렉토리 구조 확인
3. 관리자 페이지 확인
4. 글쓰기
5. 리스트
6. 글보기

##### 아나콘다에서 Django 설치하기
- 아나콘다에서 장고 프레임워크를 위해 가상환경을 만든다
$conda create --name FirstProject python=3

- 가상환경으로 들어간다
$conda activate FirstProject

- 장고를 설치한다
(FirstProject)$pip install django

(윈도우 경우 참고 사이트)
https://seogwipo.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%9E%A5%EA%B3%A0-%ED%94%84%EB%A0%88%EC%9E%84%EC%9B%8C%ED%81%AC%EC%97%90-%EC%95%B1-%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0?category=892779

#### 프로젝트 생성
- (FirstProject)$django-admin startproject tutorial
- (FirstProject)$ cd tutorial
- (FirstProject)$ -/tutorials tree(윈도우x) / tree(윈도우)
- (FirstProject)$ cd tutorial
- (FristProject)$ dir(.py 파일 조회 가능)

#### 앱 생성
(FirstProject)$ python manage.py startapp community

#### sqlite(데이터저장소 만들기 위해)
: settings.py에 있는 sqlite로 저장소 만들기 위해
- (FirstProject)$ python manage.py migrate

#### superuser 만들기
(FirstProject)$ python manage.py createsuperuser
Username : ###
Email address: ###@###.com
Password: ###

#### 홈페이지 구동
- (FristProject)$ python manage.py runserver 8080 
(기본포트 8000, 번호 바꾸고 싶으면 8080 넣기)
http://127.0.0.1:8000/
- 끌 때는 ctrl+c
+ 사이트 살펴보기
http://127.0.0.1:8000/admin
(유저 관리 가능)


#### settings.py에서 app 등록 -> 저장
Installed_Apps = [...
		'community',(쉼표 꼭!)
]

- community > models.py에 게시판 모델 class로 등록

In [ ]:
class Article(models.Model): # class는 모델을 상속
    name = models.CharField(max_length=50)
    title = models.CharField(max_length=50)
    contents = models.TextField()
    url = models.URLField()
    email = models.EmailField()
    cdate = models.DateTimeField(auto_now_add=True)

In [ ]:
저장(ctrl + s)

- community app에 수정사항이 있는지 확인 
- (FirstProject)$python manage.py makemigrations community
-> create model Article(모델 생성된 점이 수정사항이라고 뜸)

- database에 적용하려면 migrate 명령어
- (FirstProject) $python manage.py migrate
-> community란 테이블이 생성됨
-> dblite3에 만들어짐

#### 기능추가
- User들이 글 쓸 수 있도록 접근할 수 있는 url이 있어야 함
tutorials > urls.py에서 수정

In [ ]:
from django.contrib import admin
from django.urls import include, path

from community.views import *

urlpatterns = [
    path('admin/', admin.site.urls),
    path('write/', write, name='write'),
    
]

In [ ]:
# community.views에 write 함수 만들기
def write(request):
    return render(request, 'write.html')

#urls.py에서 request가 들어오면 render하는데, 'write.html' template으로
#보내는 것


In [ ]:
# community > templates폴더 > write.html파일 
<!DOCTYPE html>

<html lang="en">
<head>
        <meta charset="UTF-8">
        <title>write</title>
</head>

<body>

    hello django!
</body>
</html>

In [ ]:

# 서버 실행해보기
#(FirstProject)$ python manage.py runserver
#정상 적용 되었는지 확인

#model 내용 적용시키기 위해 community > forms.py 만듦


In [ ]:
from django.forms import ModelForm
from community.models import *

class Form(ModelForm):
    class Meta:
        model = Article
        fields=['name', 'title', 'contents', 'url', 'email']

In [ ]:
# views.py 들어가서
#새로 입력
from community.forms import *

def write(request):
    form = Form() #추가
    return render(request, 'write.html', {'form':form}) # form변수추가

In [ ]:
# write.html에서 form 그대로 가져다 쓰도록 body 수정
<body>
    <form action="" method="post">
        {% comment %} #생략가능
            {{ form.as_table }} #생략가능
            {{ form.as_p }} #생략가능
            {{ form.as_ul }} #생략가능
        {% endcomment %} #생략가능
        {{ form.as_p }}
        <button type="submit" class="btn btn-primary">저장</button>
    </form>
    
</body>


In [ ]:
# forms.py들어가서 POST일 경우 form을 그대로 DB에 저장한다
def write(request):
	if request.method == 'POST':
		form = Form(request.POST)
		if form.is_valid():
			form.save()
	else:
		form = Form() 

In [ ]:
# 메모 남겨보고, 저장하면 오류가 나는데 CSRF 토큰 미싱
-> 공격을 방어하기 위해서
write.html에서
{{ form.as_p }}
{% csrf_token %} #token 수정

# 다시 글 써보고 저장 눌러봄 (사라짐?)

#### 저장된 정보 보는 법

In [ ]:
#urls.py에서 들어가서

path('list/', list, name='list'), #  추가

In [ ]:
# views.py에 list 함수 만들고
def list(request):
    return render(request, 'list.html')

# list.html 파일 만들어서 임시로 body 수정해보고 확인

In [ ]:
# views.py 들어가서 list함수 수정
def list(request):
    articlelist = Article.objects.all()
    return render(request, 'list.html', {'articlelist':articlelist})

In [ ]:
# list.html 수정
<body>
    <ul>
        <li> 제목 | 저자 | 날짜</li>
    {% for article in articlelist %}
    <li>{{ article.title }} | {{ article.name }} | {{ article.cdate|date:"D d M Y" }}</li>
    {% endfor %}
</ul>


In [ ]:
# 다시 wrtie->list 해보면 새로운 data 저장

# 클릭하면 내용을 볼 수 있도록 수정
urls.py 에서 path('view/<inte:num>', view),
(0-9+ 숫자반복될 수 있고, num인자로 받음, view로 가라)


In [ ]:
# views.py에서 view 함수 만들기
def view(request, num='1'):
    article = Article.objects.get(id=num)
    return render(request, 'list.html', {'article':article})



In [ ]:
# view.html
<!DOCTYPE html>

<html lang="en">
<head>
        <meta charset="UTF-8">
        <title>write</title>
</head>

<body>
    <ul>
        <li> 제목 | 저자 | 날짜</li>
    {% for article in articlelist %}
    <li><a href="/view/{{article.id}}">{{ article.title }}</a> | {{ article.name }} | {{ article.cdate|date:"D d M Y" }}</li>
    {% endfor %}
</ul>
</body>
</html>

### 최종 simple 게시판 사이트 주소

http://127.0.0.1:8000/list/